In [1]:
import os
import pickle
import collections
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import sklearn.model_selection

In [2]:
data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/age/raw_data"

In [3]:
file_list = os.listdir(data_dir)
user_files = []
article_files = []
for file_name in file_list:
    if 'user_' in file_name:
        user_files.append(file_name)
    elif 'part-' in file_name:
        article_files.append(file_name)
    else:
        pass
user_files.sort()
article_files.sort()
print(user_files)
print(article_files)

['user_new']
['part-00000', 'part-00001', 'part-00002', 'part-00003', 'part-00004', 'part-00005', 'part-00006', 'part-00007', 'part-00008', 'part-00009', 'part-00010', 'part-00011', 'part-00012', 'part-00013', 'part-00014', 'part-00015', 'part-00016', 'part-00017', 'part-00018', 'part-00019', 'part-00020', 'part-00021', 'part-00022', 'part-00023', 'part-00024', 'part-00025', 'part-00026', 'part-00027', 'part-00028', 'part-00029', 'part-00030', 'part-00031', 'part-00032', 'part-00033', 'part-00034', 'part-00035', 'part-00036', 'part-00037', 'part-00038', 'part-00039', 'part-00040', 'part-00041', 'part-00042', 'part-00043', 'part-00044', 'part-00045', 'part-00046', 'part-00047', 'part-00048', 'part-00049', 'part-00050', 'part-00051', 'part-00052', 'part-00053', 'part-00054', 'part-00055', 'part-00056', 'part-00057', 'part-00058', 'part-00059', 'part-00060', 'part-00061', 'part-00062', 'part-00063', 'part-00064', 'part-00065', 'part-00066', 'part-00067', 'part-00068', 'part-00069', 'part-

In [4]:
%%time
all_file_data = []
for file_name in user_files:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None))
user_table = pd.concat(all_file_data,axis=0,ignore_index=True)

user_table.columns = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲']
# 获得指定列
useful_user_df = user_table[["用户id","用户姓名","用户出生时间","微信网名","关注的作者","关注的马甲"]]
del all_file_data
del user_table
useful_user_df

CPU times: user 3.62 s, sys: 245 ms, total: 3.87 s
Wall time: 3.87 s


,用户id,用户姓名,用户出生时间,微信网名,关注的作者,关注的马甲
0,00001692e9a0e16ce869b88edaf023a9db8e61d0,-1,0-0-0,-1,"1534653,1870542","华夏基石e洞察,caoz的梦呓©"
1,00001D26FE991A4BC4A01325D5E222DE,0,-1,"0,1",-1,-1
2,00008225fde7a2d26dd0e7b770cb6de22e92654d,-1,0-0-0,-1,-1,-1
3,0000BE70BB0233665552E7C1AC45DDA7,0,-1,"0,1",-1,-1
4,0000d2529e07eb6355a786fa03d7db67f72cccc6,0,-1,"0,1",-1,-1
...,...,...,...,...,...,...
2738209,qlu2hw9gd45m34i3zgyd,0,-1,"0,1",-1,-1
2738210,qlu2mrq87mtpyzerr0s,0,-1,"0,1",-1,-1
2738211,qlu2n729ejbhpp59tsq,0,-1,"0,1",-1,-1
2738212,qlu2xlbspo1bw8wwujh,-1,1-1-1,"0,1",-1,-1


## 数据分析

In [5]:
temp_list = []
for i in useful_user_df['用户出生时间']:
    if (not i.split("-")[0]) or (int(i.split("-")[0]) < 1940):
        temp_list.append(i)
len(temp_list)

2735998

In [6]:
collections.Counter(temp_list)

Counter({'0-0-0': 169581,
         '-1': 1812577,
         '1-1-1': 753832,
         '0-10-0': 1,
         '0-5-10': 1,
         '0-12-7': 1,
         '0-3-0': 1,
         '0-4-25': 1,
         '0-11-28': 1,
         '0-4-13': 1,
         '0-2-17': 1})

In [7]:
def deal_with_birth_date(birth_date):
    if (not birth_date.split("-")[0]) or (int(birth_date.split("-")[0]) < 1940):
        return False
    return True

useful_user_df = useful_user_df[useful_user_df["用户出生时间"].apply(deal_with_birth_date)]

In [8]:
useful_user_df

,用户id,用户姓名,用户出生时间,微信网名,关注的作者,关注的马甲
609,007DFA864B32EBC0280192C72438DB89,春秋,1991-9-26,-1,2061955,我不叫塞尔达
829,00B4D09FC36D629E3249D55B59813BB3,贾博鑫,1985-11-24,hyacinthusboy,1342281,周超臣
974,00E80E5F9B2EF6EDA96AD73A3257B27E,-1,1990-11-14,-1,-1,-1
990,00ECCE733FA3664C620DD1515DC24D78,-1,1983-6-1,jun0195,"1390243,456826,456836,2392109,456869,2061955,4...","吴倩男,风向·标,吸睛收购案,钛禾产业观察,-1,我不叫塞尔达,互联网指北,区块链动态,侯宏..."
1547,013795e3bf12565ea7f8c179806e633d5965664e,海深蓝,1985-1-7,-1,"456875,500260,500259","百度又有什么新动作,头条表示不服,美团在干嘛？"
...,...,...,...,...,...,...
2728611,qiwgvftlwg277kr6fc,-1,1991-3-8,"0,1,-1",-1,-1
2731699,qjswu7xcqfs2blyua7h,扶嵩,1983-1-1,"0,1,-1",-1,-1
2732320,qk2lzj8owr81hz81fan8,-1,1993-12-18,ydyd51,-1,-1
2733786,qkil95x7vd15adb5a4lz,吴存志,1978-8-12,-1,-1,-1


In [9]:
#查看信息
useful_user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 609 to 2734452
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   用户id    2216 non-null   object
 1   用户姓名    2216 non-null   object
 2   用户出生时间  2216 non-null   object
 3   微信网名    2216 non-null   object
 4   关注的作者   2216 non-null   object
 5   关注的马甲   2216 non-null   object
dtypes: object(6)
memory usage: 121.2+ KB


In [10]:
useful_user_df['用户姓名'].unique()

array(['春秋', '贾博鑫', '-1', ..., '刘宇', '扶嵩', '吴存志'], dtype=object)

In [11]:
(useful_user_df['用户姓名']=="-1").sum()

592

In [12]:
#男，女，默认，缺失 -1
useful_user_df['用户出生时间'].unique()

array(['1991-9-26', '1985-11-24', '1990-11-14', ..., '1991-3-8',
       '1978-8-12', '1992-11-15'], dtype=object)

In [13]:
useful_user_df['微信网名'].unique()

array(['-1', 'hyacinthusboy', 'jun0195', 'leo6543210', '-1,0,1',
       'gl369723', '18817950617', '1301356174', 'kxr1983', 'shui620697',
       '529804222', 'OCDer_Shawn', '微信公众号：睡后故事', 'normwang', 'harryyao',
       '18917568663', 'ken8119', 'YCH-107', 'libin65316', 'warlockcp',
       'myucity', 'sdzcjfc', 'zhaoying933', 'fu_yongcheng', 'teviky',
       'bourne0214', 'ancheng365', 'Mr_yang0417', 'tweibo999',
       'eaF1菜老湿,0,1', 'leo3916622', 'netmedia', 'scenario2012',
       'lanyuemoshang', '18616770810', 'genaa19', 'SJ865613809', 'zhleao',
       '188009827254', 'sasithara22', 'allenspassion', '0,1,a736964479',
       'shuidam999', 'canghai', 'zzt8889966', 'li1156119310',
       'muqin0512,-1,0,1', '0,1,61386563', '478164932', 'dinglvyu',
       'llpdzh', 'wukongzhao', '微信号：seovsem  微信公众号：刘海政的大搜罗', '狗王的每日思考',
       'oneisall2014', 'riverhe', '0,1,-1', 'ideaone,0,1', 'hongyanweiyu',
       'sxczhdcj', 'kefu_ylzbl', 'abcde7386', 'wen-wu-zhao', '公众号-青年投资人',
       'xm648931554', 

In [14]:
useful_user_df['关注的作者'].unique()

array(['2061955', '1342281', '-1',
       '1390243,456826,456836,2392109,456869,2061955,456858,1435071,500237,64711,500123,456883,1585427,2480289,456857,2321658,2381804,456827,456867,1640668,500255',
       '456875,500260,500259',
       '1534653,2400280,2660070,2067736,2359462,2195474,1535828,2457173,1872067,2172501,2213821,2482066',
       '456875',
       '2047693,2038137,456826,1872067,1871615,2365822,500276,2195090,2373758,2648484,500274,2309344,1543372,2410540,2479505,2650244,303086,1871617,2528929,2147218,1870542',
       '10140,1816236,1546218,1932107,19,400003,2327553,2480049,1494345,1854035,88973,1598894,2515836,10246,2047286,1890151,2480289,2733254,2616820,1330066,2590757,1871696,2458922,277654,2026620,2405867,2581735,303086,134,174882,1785733,1843091,164,2199899,1390243,31445',
       '500145,1342281,2684393,456855,400008,500246,456866,500274,500253,456857,1959295,456827',
       '1453857,1871664', '1435071,1598894',
       '2080977,2016550,2068342,1992309,1890151', '140,13

In [15]:
(useful_user_df['关注的作者']=="-1").sum()

1069

In [16]:
temp_list = []
for _str in useful_user_df['关注的作者'].fillna(""):
    for _id in _str.split(","):
        temp_list.append(_id)

In [17]:
collections.Counter(temp_list)

Counter({'2061955': 30,
         '1342281': 93,
         '-1': 1278,
         '1390243': 87,
         '456826': 144,
         '456836': 24,
         '2392109': 14,
         '456869': 38,
         '456858': 65,
         '1435071': 24,
         '500237': 8,
         '64711': 18,
         '500123': 31,
         '456883': 63,
         '1585427': 9,
         '2480289': 64,
         '456857': 25,
         '2321658': 3,
         '2381804': 1,
         '456827': 70,
         '456867': 27,
         '1640668': 52,
         '500255': 17,
         '456875': 50,
         '500260': 29,
         '500259': 23,
         '1534653': 19,
         '2400280': 6,
         '2660070': 13,
         '2067736': 8,
         '2359462': 9,
         '2195474': 13,
         '1535828': 26,
         '2457173': 11,
         '1872067': 60,
         '2172501': 70,
         '2213821': 13,
         '2482066': 12,
         '2047693': 51,
         '2038137': 8,
         '1871615': 66,
         '2365822': 37,
         '500276':

In [18]:
#查看行业信息
useful_user_df['关注的马甲'].unique()

array(['我不叫塞尔达', '周超臣', '-1',
       '吴倩男,风向·标,吸睛收购案,钛禾产业观察,-1,我不叫塞尔达,互联网指北,区块链动态,侯宏剑桥杂思,老·司机,读数,出类公司,fanglopez,物流行业动态,网易研究局©,嗅友itdvv,投融资捕手,新政新规,皮特王新经营哲学,科学咖',
       '百度又有什么新动作,头条表示不服,美团在干嘛？',
       '华夏基石e洞察,新金融洛书,盖茨基金会©,Vista看天下©,我是科学家iScientist,真实故事计划©,一本财经,投中网,果壳©,看理想©,起朱楼宴宾客©,原理©',
       '百度又有什么新动作',
       '大家·腾讯新闻©,阿尔法工场研究院,风向·标,果壳©,腾讯新闻棱镜深网,国泰君安证券研究©,股市短评,敲敲格,真叫卢俊©,远川科技评论©,重磅,混子谈钱,苏宁金融研究院,叁里河©,智本社,牛顿顿顿©,Eastland,腾讯科技©,界面教育©,经济观察报观察家©,caoz的梦呓©',
       '李清乐,古泉君,司马徐龙,刘然,虎嗅,阿里经济体,李国飞哲学与投资笔记©,互联网怪盗团,尹生,深响,新语©蜜蜂观察,人神共奋,孙鸣远,魏武挥,商业数据派,饭统戴老板,fanglopez,虎嗅投研,杨旭然©,秋源俊二,陈闷雷,财经杂志©,桑明强,朱思码记,格隆汇,刘 宇 豪,最人物©,Eastland,-1,评论尸,mrpuppybunny,略大参考,Cuba Libre,吴倩男,俊世太保',
       '曹会计,周超臣,美国疾病控制与预防中心©,微信动向,影君子,货币的战争,漫步太空,重磅,华为稳不稳,物流行业动态,商业纪事,投融资捕手',
       '薛洪言,许小年©', '互联网指北,人神共奋', '8字路口,蹦迪班长,北方公园NorthPark,-1,饭统戴老板',
       '-1,吴倩男',
       'IPO动态,金融圈,周超臣,小檠,曹会计,华尔街见闻©,传言,突发,fanglopez,王兴这么说,阿里经济体,衣公子的剑,物流行业动态,有故事的董小姐,耍大牌儿,美团在干嘛？,风向·标,这话我服,吸睛收购案,Eastland,李小猫不爱吃鱼,果粉er,-1,小晚wan,驿站老鬼,敲敲格',
       '腾讯研究院,大开

In [19]:
(useful_user_df['关注的马甲']=="-1").sum()

1086

In [20]:
%%time
all_file_data = []
for file_name in article_files:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None))
article_table = pd.concat(all_file_data,axis=0,ignore_index=True)

article_table.columns = ['用户id','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
# 获得指定列
useful_article_data = article_table[['用户id','文章频道','浏览文章时间','文章类型','文章图片数量','文章字数']]
del all_file_data
del article_table
useful_article_data

CPU times: user 54 s, sys: 2.44 s, total: 56.5 s
Wall time: 56.5 s


,用户id,文章频道,浏览文章时间,文章类型,文章图片数量,文章字数
0,002947a7781e89bffdafcad888898e2a73594747,"-1,金融地产,十亿消费者,财经,财经,金融地产,十亿消费者","2020-07-22 17:21:01,2020-07-16 08:35:38,2020-0...","-1,乐视","-1,1","-1,<p><span class=""text-remarks"" label=""备注"">本文..."
1,00297F3B00C4CD076C22FA29796929D1,"-1,文化教育,生活腔调,生活腔调,文化教育","2020-06-20 09:46:13,2020-06-20 09:33:22,2020-0...",-1,"-1,2","<p><span class=""text-remarks"" label=""备注"">本文来自微..."
2,006431b78c728426b28ed16371ee105b5aad5adf,-1,"2020-07-19 13:10:39,2020-06-30 10:26:58,2020-0...",-1,-1,-1
3,006A3DEA36EB70939ED2E958BE24090A,"-1,全球热点,出海,娱乐淘金,社交通讯,社交通讯,娱乐淘金,年轻一代","2020-07-14 12:20:38,2020-06-24 10:38:19,2020-0...","-1,创投,大公司,软件应用,航空航天","17,-1,3,12,5,8","-1,<p><span class=""text-remarks"" label=""备注""></..."
4,00A099F603433E280303E4253B501BE9,"十亿消费者,-1","2020-06-07 11:48:26,2020-06-07 11:43:16,2020-0...","-1,商业模式","13,-1,14","-1,<p><span class=""text-remarks"" label=""备注"">本文..."
...,...,...,...,...,...,...
203997,phojzwrkcm2ck4i9rw4w,"-1,医疗健康","2020-07-23 15:52:01,2020-07-29 18:14:29,2020-0...","军事政治,空,-1","1,9,-1","-1,<p>一直以来，人大代表建议和政协委员提案都是每年两会的重要成果，亦是传递民众关切、吸..."
203998,ppnlvxat3q36kbnbc3wa,-1,"2020-06-16 04:03:55,2020-06-16 04:03:54,2020-0...","商业模式,金融其他,-1","1,8,11,7,-1",<p>&nbsp;7年来，生鲜零售领域城头变幻大王旗，你方唱罢我方登场，两三年就换一波新的挑...
203999,q2lo39n54la38fftf6,-1,"2020-07-31 09:13:32,2020-07-31 09:12:57",-1,-1,-1
204000,q6jcef3gusgdf8tghfu,-1,2020-07-27 09:36:44,-1,-1,-1


In [21]:
useful_article_data["文章频道"].unique()

array(['-1,金融地产,十亿消费者,财经,财经,金融地产,十亿消费者', '-1,文化教育,生活腔调,生活腔调,文化教育', '-1',
       ..., '-1,十亿消费者,金融地产,金融地产,十亿消费者,年轻一代,暂未归类,金融地产',
       '娱乐淘金,十亿消费者,年轻一代,娱乐淘金,年轻一代,-1,视频,医疗健康,全球热点,娱乐淘金,年轻一代,十亿消费者,视频',
       '全球热点,金融地产,24小时,车与出行,-1,视频,金融地产,全球热点,金融地产,车与出行,十亿消费者,文化教育'],
      dtype=object)

In [22]:
temp_list = []
for _str in useful_article_data["文章频道"].fillna(""):
    for v in _str.split(","):
        temp_list.append(v)
collections.Counter(temp_list)

Counter({'-1': 199782,
         '金融地产': 54742,
         '十亿消费者': 98282,
         '财经': 35600,
         '文化教育': 60171,
         '生活腔调': 52870,
         '全球热点': 30935,
         '出海': 27708,
         '娱乐淘金': 65976,
         '社交通讯': 23977,
         '年轻一代': 66044,
         '创业维艰': 39935,
         '企业服务': 21475,
         '视频': 18194,
         '前沿科技': 46947,
         '车与出行': 28879,
         '医疗健康': 18675,
         '暂未归类': 3166,
         '24小时': 5294,
         '突发爆料': 190,
         '人工智能': 87})

In [23]:
temp_list = []
for _str in useful_article_data["文章类型"].fillna(""):
    for v in _str.split(","):
        temp_list.append(v)
collections.Counter(temp_list)

Counter({'-1': 199915,
         '乐视': 301,
         '创投': 12211,
         '大公司': 24738,
         '软件应用': 11712,
         '航空航天': 7538,
         '商业模式': 14395,
         '政策': 2294,
         '历史人文': 1007,
         '大数据与云': 3227,
         '共享经济': 1686,
         '文娱其它': 12816,
         '数码硬件其他': 5423,
         '快递': 2265,
         '传媒': 4599,
         '酒与美食': 8833,
         '地产': 8450,
         '空': 23798,
         '科普': 2736,
         '电商': 11272,
         '教育': 8855,
         '汽车': 11066,
         '社会': 6214,
         '神州优车': 3771,
         '零售与消费其他': 4201,
         '手机': 1689,
         '年轻人': 8127,
         '旅游': 137,
         '医疗': 9884,
         '互联网金融': 369,
         '股市': 2527,
         '家居': 171,
         '阿里': 1697,
         '生活方式其他': 6685,
         '交通': 1271,
         '军事政治': 2135,
         '社交': 2638,
         '职场': 1475,
         '文化其他': 231,
         '影音剧': 6344,
         '全球': 3078,
         '美团': 701,
         '前沿技术其他': 3399,
         '[]': 182,
         '通用电气': 3,
        

In [24]:
user_df = useful_user_df.merge(useful_article_data,how="left")
del useful_user_df
del useful_article_data

In [25]:
user_df.fillna("",inplace=True)

In [48]:
user_df["文章字数"][0]

'-1,<p><span class="text-remarks" label="备注">出品丨虎嗅「要玩儿」节目组</span></p><p><font color="#999999"><span style="caret-color: rgb(153, 153, 153);">文案</span></font><span style="caret-color: rgb(153, 153, 153); color: rgb(153, 153, 153);">丨曾欢</span></p><p><font color="#999999"><span style="caret-color: rgb(153, 153, 153);">视频监制｜张博文</span></font></p><p><font color="#999999"><span style="caret-color: rgb(153, 153, 153);">视频制作｜狄枭 韩子涵 葛佳澍</span></font></p><p><br/></p><p>成长有两种方式，一种是年龄与想象力逐渐变成零和程序，年龄的增长无情地消磨尽少年的憧憬；另一种则是积累的阅历和更多的好奇心，将年龄按入时空的循环，让每个时点都成了起点，从此人生的质感无关沧桑岁月。</p><p><br/></p><p>是的，年龄像风，它熄灭以第一种方式成长之人的童心，却也能使后者的童心越烧越旺。今天是六一，除了祝福孩子，我们也想聊聊以第二种方式成长的大人。</p><p><br/></p><p>有这样两位大叔，他们恰巧分别年过半百，却都在中年时，用“玩具”为自己重建了时间。</p><p><br/></p><p class="img-center-box"><img src="https://img.huxiucdn.com/article/content/202006/01/142756561057.jpg?imageView2/2/w/1000/format/jpg/interlace/1/q/85" data-w="1000" data-h="667"/></p><p><br/></p><p>许群，人称“四万大叔”，1969年生人，资深汽车媒体人，年轻的时候在德国学习和生活。关于他，广为人知的几个事实是：他在北京机场高速边上租下400平米的空

In [27]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 0 to 2215
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   用户id    2216 non-null   object
 1   用户姓名    2216 non-null   object
 2   用户出生时间  2216 non-null   object
 3   微信网名    2216 non-null   object
 4   关注的作者   2216 non-null   object
 5   关注的马甲   2216 non-null   object
 6   文章频道    2216 non-null   object
 7   浏览文章时间  2216 non-null   object
 8   文章类型    2216 non-null   object
 9   文章图片数量  2216 non-null   object
 10  文章字数    2216 non-null   object
dtypes: object(11)
memory usage: 207.8+ KB


In [ ]:
# user_df = user_df[['用户id','用户出生时间','关注的作者','关注的马甲','文章频道','浏览文章时间','文章类型','文章图片数量','文章字数']]
# user_df

In [49]:
#生成multi_hot_dict
multi_hot_dict = {}
multi_hot_dict["关注的作者"] = {}
multi_hot_dict["关注的马甲"] = {}
multi_hot_dict["文章频道"] = {}
multi_hot_dict["文章类型"] = {}

temp_list = []
for _str in user_df['关注的作者']:
    for _id in _str.split(","):
        temp_list.append(_id)
for i,k in enumerate(collections.Counter(temp_list)):
    multi_hot_dict["关注的作者"][k] = i

temp_list = []
for _str in user_df['关注的马甲']:
    for _id in _str.split(","):
        temp_list.append(_id)
for i,k in enumerate(collections.Counter(temp_list)):
    multi_hot_dict["关注的马甲"][k] = i
    
temp_list = []
for _str in user_df['文章频道']:
    for _id in _str.split(","):
        temp_list.append(_id)
for i,k in enumerate(collections.Counter(temp_list)):
    multi_hot_dict["文章频道"][k] = i

temp_list = []
for _str in user_df['文章类型']:
    for _id in _str.split(","):
        temp_list.append(_id)
for i,k in enumerate(collections.Counter(temp_list)):
    multi_hot_dict["文章类型"][k] = i

temp_list = []
for _str in user_df['文章类型']:
    for _id in _str.split(","):
        temp_list.append(_id)
for i,k in enumerate(collections.Counter(temp_list)):
    multi_hot_dict["文章类型"][k] = i    
    


for k,v in multi_hot_dict.items():
    print(k,len(v))

关注的作者 2359
关注的马甲 2217
文章频道 22
文章类型 122


In [50]:
# #生成multi_hot_dict

# def genereate_split_comma_column_multi_hot(column,user_df,multi_hot_dict):
#     temp_list = []
#     multi_hot_dict[column] = {}
#     for _str in user_df[column]:
#         for _id in _str.split(","):
#             temp_list.append(_id)
#     for i,k in enumerate(collections.Counter(temp_list)):
#         multi_hot_dict[column][k] = i
#     return multi_hot_dict

# multi_hot_dict = {}
# for column in ["关注的作者",'关注的马甲','文章频道',"文章类型"]:
#     multi_hot_dict = genereate_split_comma_column_multi_hot(column,user_df,multi_hot_dict)

# for k,v in multi_hot_dict.items():
#     print(k,len(v))

In [71]:
import datetime
x_dict = {}
y_dict = {}
x_predict = {}
# 1-20岁
# 20-25岁
# 26-29岁
# 30-35岁
# 40-45岁
# 46-50岁
# 50以上
def bucket_age(age):
    category = 7
    y_list = [0 for _ in range(category)]
    if age < 20:
        y_list[0] = 1
    elif age < 25:
        y_list[1] = 1
    elif age < 29:
        y_list[2] = 1
    elif age < 35:
        y_list[3] = 1
    elif age < 45:
        y_list[4] = 1
    elif age < 50:
        y_list[5] = 1
    else:
        y_list[6] = 1
    return y_list

In [72]:
df = user_df
    
for index in df.index:
    birth_date = df.loc[index,"用户出生时间"]
    user_id = df.loc[index,"用户id"]
    
    y_dict[user_id] = bucket_age(
        datetime.datetime.now().year - int(birth_date.split("-")[0])
    )
    
    x_list = []

    
    temp_str = df.loc[index,"关注的作者"]
    temp_list = [0 for _ in range(len(multi_hot_dict["关注的作者"]))]
    for v in temp_str.split(","):
        temp_list[multi_hot_dict["关注的作者"][v]] += 1
    x_list.extend(temp_list)
    #x_list.append(hash("".join([str(v) for v in temp_list]))%20)
    #此处  hash不hash  差别不大

    temp_str = df.loc[index,"关注的马甲"]
    temp_list = [0 for _ in range(len(multi_hot_dict["关注的马甲"]))]
    for v in temp_str.split(","):
        temp_list[multi_hot_dict["关注的马甲"][v]] += 1
    x_list.extend(temp_list)
    #x_list.append(hash("".join([str(v) for v in temp_list]))%20)

    temp_str = df.loc[index,"文章频道"]
    temp_list = [0 for _ in range(len(multi_hot_dict["文章频道"]))]
    for v in temp_str.split(","):
        temp_list[multi_hot_dict["文章频道"][v]] += 1
    x_list.extend(temp_list)
    #x_list.append(hash("".join([str(v) for v in temp_list]))%100)

    temp_str = df.loc[index,"文章类型"]
    temp_list = [0 for _ in range(len(multi_hot_dict["文章类型"]))]
    for v in temp_str.split(","):
        temp_list[multi_hot_dict["文章类型"][v]] += 1
    x_list.extend(temp_list)
    #x_list.append(hash("".join([str(v) for v in temp_list]))%100)
    
    x_dict[user_id] = x_list
    

In [73]:
x_list = []
y_list = []
for user_id,temp_list in x_dict.items():
    x_list.append(temp_list)
    y_list.append(y_dict[user_id])

In [74]:
x_npa = np.array(x_list)
y_npa = np.array(y_list)

In [75]:
x_npa.shape

(2216, 4720)

In [76]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x_npa, y_npa, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [79]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(4720,)) )
model.add(tf.keras.layers.Dense(7, activation='softmax',use_bias=True,
                                #kernel_regularizer=tf.keras.regularizers.l1(0.01)
                               ))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.binary_crossentropy, 
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [84]:
model.fit(x_train, 
          y_train, 
          batch_size=64,
          epochs=20,
         validation_data = (x_valid,y_valid)
         )

Epoch 1/20
21/21 [==============================] - 0s 3ms/step - loss: 0.2325 - binary_accuracy: 0.9085 - val_loss: 0.3162 - val_binary_accuracy: 0.8771
Epoch 2/20
21/21 [==============================] - 0s 2ms/step - loss: 0.2317 - binary_accuracy: 0.9084 - val_loss: 0.3162 - val_binary_accuracy: 0.8771
Epoch 3/20
21/21 [==============================] - 0s 3ms/step - loss: 0.2308 - binary_accuracy: 0.9092 - val_loss: 0.3162 - val_binary_accuracy: 0.8771
Epoch 4/20
21/21 [==============================] - 0s 3ms/step - loss: 0.2299 - binary_accuracy: 0.9092 - val_loss: 0.3162 - val_binary_accuracy: 0.8771
Epoch 5/20
21/21 [==============================] - 0s 2ms/step - loss: 0.2291 - binary_accuracy: 0.9098 - val_loss: 0.3163 - val_binary_accuracy: 0.8771
Epoch 6/20
21/21 [==============================] - 0s 2ms/step - loss: 0.2283 - binary_accuracy: 0.9101 - val_loss: 0.3164 - val_binary_accuracy: 0.8775
Epoch 7/20
21/21 [==============================] - 0s 2ms/step - loss: 0.22

In [85]:
model.evaluate(x_test,y_test)

14/14 [==============================] - 0s 1ms/step - loss: 0.3187 - binary_accuracy: 0.8758


[0.31869834661483765, 0.8758043646812439]